In [1]:
import numpy as np
import scipy.linalg as la
from scipy.linalg import expm
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor, as_completed
import multiprocessing
from Qdrift import evolve_Qdrift_trajectories


In [2]:
#1D Hamiltonian definitions
X = np.array([[0.,1.],[1.,0.]],dtype=complex)
Y = np.array([[0.,-1.j],[1.j,0.]],dtype=complex)
Z = np.array([[1.,0.],[0.,-1.]],dtype=complex)
I = np.array([[1.,0.],[0.,1.]],dtype=complex)


def define1D_Hterm(nsites,site,pauli,twosites=False):
    #calculates matrix of a single Hamiltonian term with spectral norm=1
    #nsites: number of spin 1/2 sites
    #site: site index of the term Hi
    #pauli: can be X Y Z for site i
    #twosites: False=> site i, True=> site i and i+1 
    op_list = [I] * nsites
    if not twosites:
        op_list[site]= pauli
    else:
        op_list[site]= pauli
        op_list[(site+1)%nsites] = pauli

    result = op_list[0]
    for op in op_list[1:]:
        result = np.kron(result, op)
    return result

def define1D_Hamiltonian(nsites,terms,coefficients,d=2):
    #dictionary containing all hamiltonian information and initial state
    Hdict=dict()
    Hdict["nsites"]=nsites
    Hdict["local_dim"]=d
    Hdict["hilbert_dim"]=d**nsites
    Hdict["terms"]=terms
    Hdict["coefficients"]=np.array(coefficients)
    Hdict["lambda"]=sum(coefficients)
    Hdict["probabilities"]=np.abs(Hdict["coefficients"]/Hdict["lambda"])
    Hdict["weighted_terms"]=[hi*Hi for hi,Hi in zip(coefficients,terms)]
    Hdict["Hamiltonian"]=sum(Hdict["weighted_terms"])
    
    # initial updownupdown state
    assert d==2 # if later we want to do qtrits ?
    binary_str = ''.join(['0' if i % 2 == 0 else '1' for i in range(nsites)])
    state_index = int(binary_str, 2)
    state_vector = np.zeros(2**nsites, dtype=complex)
    state_vector[state_index] = 1
    Hdict["initial_state"]=state_vector
    return Hdict

def Ising1D_Hamiltonian(nsites,J=1,h=1,periodic=False):

    if periodic: 
        p=0 
    else: 
        p=1

    Jterms= [define1D_Hterm(nsites,i,Z,twosites=True) for i in range(nsites-p)]
    hterms= [define1D_Hterm(nsites,i,X) for i in range(nsites)]
    Jcoefs= [-J]*len(Jterms)
    hcoefs= [-h]*len(hterms)

    return define1D_Hamiltonian(nsites,Jterms+hterms,Jcoefs+hcoefs)



In [3]:
# Time evolution


#Exact evolution
def evolve_exact(Hdict,total_time,initial_state=None):
    if initial_state is None:
        initial_state=Hdict["initial_state"]
    H=Hdict["Hamiltonian"]

    return la.expm(-1.j*total_time*H)@initial_state

#Trotter evolution
def evolve_Trotter(Hdict,total_time,nsteps,initial_state=None):
    if initial_state is None:
        initial_state=Hdict["initial_state"]
    
    dt=total_time/nsteps
    evolved_state=initial_state.copy()
    gates=[]
    for weighted_Hterm in Hdict["weighted_terms"]:
        gates.append(expm(-1j * dt * weighted_Hterm))
    
    for _ in range(nsteps):
        for gate in gates:
            evolved_state= gate @ evolved_state

    return evolved_state

#Qdrift evolution
# in Qdrift.py for ProcessPoolExecutor


In [4]:
# Compute multiple evolutions

def compute_Qdrift_evolutions(Hdict,total_time_list,nsamples_list,ntrajs,initial_state=None):
    # total times list x nsamples list x trajectories x 2**N
    qstates=np.zeros((len(total_time_list),len(nsamples_list),ntrajs,Hdict["hilbert_dim"]),dtype=complex)
    with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
        futures = []
        for total_time_idx, total_time in enumerate(total_time_list):
            for nsamples_idx, nsamples in enumerate(nsamples_list):
                #qstates[total_time_idx,nsamples_idx]=evolve_Qdrift_trajectories(Hdict,total_time,nsamples,ntrajs,initial_state)
                futures.append(executor.submit(evolve_Qdrift_trajectories,Hdict,total_time,nsamples,ntrajs,initial_state,(total_time_idx,nsamples_idx)))

            for future in as_completed(futures):
                evolved_trajs,idx=future.result()
                qstates[idx[0],idx[1]]=evolved_trajs
    return qstates


In [5]:
ising1D=Ising1D_Hamiltonian(3)

qstates=compute_Qdrift_evolutions(ising1D,[1,2,3],[100,200],10)

NameError: name 'la' is not defined

In [30]:
qstates.shape

(3, 2, 10, 8)